In [1]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
size_cloth_column = ['newClothId', 'clothMale', 'clothFemale', 'clothRate', 'clothReviewCnt', 'userHeight', 'userWeight', 'userMale', 'userFemale']
size_user_column = ['userId','userMale', 'userFemale', 'userHeight', 'userWeight', 'clothCnt']
favor_ctop_column = ['newClothId', 'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
        'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'userMale', 'userFemale', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_couter_column = ['newClothId', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cpants_column = ['newClothId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_conepiece_column = ['newClothId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cskirt_column = ['newClothId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_utop_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'touch', 'smallCategoryHalfshort', 'smallCategoryShirt',
       'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit', 'smallCategoryLong', 'smallCategoryShort',
       'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12', 'clothCnt']
favor_uouter_column = ['userId', 'size', 'bright', 'color', 'thickness',
       'weightness', 'touch', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']
favor_upants_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12', 'clothCnt']
favor_uonepiece_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']
favor_uskirt_column = ['userId', 'size', 'bright', 'color', 'thickness', 'weightness', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12', 'clothCnt']

In [4]:
transaction = pd.read_json('./transaction_skirt.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]
transaction

,userId,newClothId,shopCnt
0,0,19641661812,1
1,1,1795772837,1
2,1,21172562649,1
3,2,20995251016,1
4,3,19136511221,1
...,...,...,...
10306,8292,15601831248,2
10307,8293,2214923407,1
10308,8294,20462111605,1
10309,8295,21014592225,1


In [5]:
user_meta = pd.read_json('./user_skirt.json')
cloth_meta = pd.read_json('./final_skirt.json')

In [6]:
user_meta = user_meta[size_user_column]
user_meta

,userId,userMale,userFemale,userHeight,userWeight,clothCnt
0,0,0,1,160,55,1
1,1,0,1,160,49,2
2,2,0,1,155,50,1
3,3,0,1,161,44,1
4,4,0,1,164,47,4
...,...,...,...,...,...,...
8292,8292,0,1,157,55,2
8293,8293,0,1,152,45,1
8294,8294,0,1,169,48,1
8295,8295,0,1,160,48,1


In [7]:
cloth_meta = cloth_meta[size_cloth_column]
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale
0,2018790,0,1,0.0,0,0.0,0,0.0,1.0
1,1844574,0,1,0.0,0,0.0,0,0.0,1.0
2,1672093,0,1,0.0,0,0.0,0,0.0,1.0
3,1687533,0,1,0.0,0,0.0,0,0.0,1.0
4,2338446,0,1,0.0,0,0.0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
8871,21172562649,0,1,4.8,148,165.0,55,0.0,1.0
8872,21172562648,0,1,4.8,148,175.0,65,0.0,1.0
8873,23400702652,0,1,4.7,14,160.0,55,0.0,1.0
8874,23400702653,0,1,4.7,14,161.0,57,0.0,1.0


In [8]:
unique_cloth_id = list(transaction['newClothId'].unique())

In [9]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))
cloth_meta

,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale
0,161812522,0,1,4.6,10,165.000000,54,0.0,1.0
1,161812521,0,1,4.6,10,162.000000,60,0.0,1.0
2,185310349,0,1,5.0,1,167.333333,57,0.0,1.0
3,1884867110,0,1,5.0,2,167.000000,56,0.0,1.0
4,1629303146,0,1,4.8,16,162.000000,44,0.0,1.0
...,...,...,...,...,...,...,...,...,...
2649,21172562650,0,1,4.8,148,161.000000,46,0.0,1.0
2650,21172562649,0,1,4.8,148,165.000000,55,0.0,1.0
2651,21172562648,0,1,4.8,148,175.000000,65,0.0,1.0
2652,23400702652,0,1,4.7,14,160.000000,55,0.0,1.0


In [10]:
transaction_source = [(transaction['userId'][i], transaction['newClothId'][i]) for i in range(transaction.shape[0])]
transaction_source

[(0, 19641661812),
 (1, 1795772837),
 (1, 21172562649),
 (2, 20995251016),
 (3, 19136511221),
 (4, 179576768),
 (4, 1969751296),
 (4, 20653811901),
 (4, 22305121572),
 (5, 17341611387),
 (6, 22532111930),
 (7, 2287002788),
 (8, 1888732235),
 (9, 21956201970),
 (10, 1173615491),
 (11, 18865111957),
 (12, 1324169973),
 (13, 15601831248),
 (13, 15601831249),
 (14, 1901383986),
 (15, 1324169974),
 (16, 11259151759),
 (16, 21172562650),
 (17, 12308491709),
 (18, 16599981671),
 (18, 17341611387),
 (19, 21956201971),
 (20, 18568061391),
 (21, 19113212245),
 (22, 2231958521),
 (23, 14474156),
 (24, 22224141751),
 (25, 86558902567),
 (26, 17888651479),
 (26, 19208661679),
 (27, 19178321222),
 (28, 21172511666),
 (29, 16422201302),
 (30, 1618698367),
 (31, 2214923407),
 (32, 18253822156),
 (33, 15601831248),
 (34, 19318941269),
 (34, 21172562649),
 (35, 1247689946),
 (36, 11736131636),
 (37, 20444262358),
 (38, 18391891912),
 (39, 21172521727),
 (40, 21658241749),
 (41, 2143308253),
 (42, 114858

In [11]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.newClothId.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.newClothId.unique()))

userId의 유니크 개수 : 8297
clothId의 유니크 개수 : 2654
user테이블의 userId의 유니크 개수 : 8297
cloth테이블의 clothId의 유니크 개수 : 2654


In [12]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i], user_meta['clothCnt'][i]]) for i in range(user_meta.shape[0])]
user_features_source

[(0, [0, 1, 160, 55, 1]),
 (1, [0, 1, 160, 49, 2]),
 (2, [0, 1, 155, 50, 1]),
 (3, [0, 1, 161, 44, 1]),
 (4, [0, 1, 164, 47, 4]),
 (5, [0, 1, 162, 57, 2]),
 (6, [0, 1, 163, 49, 1]),
 (7, [0, 1, 161, 45, 1]),
 (8, [0, 1, 166, 53, 1]),
 (9, [0, 1, 158, 44, 1]),
 (10, [0, 1, 162, 60, 1]),
 (11, [0, 1, 162, 53, 1]),
 (12, [0, 1, 158, 60, 1]),
 (13, [0, 1, 162, 52, 3]),
 (14, [0, 1, 156, 42, 1]),
 (15, [0, 1, 161, 42, 1]),
 (16, [0, 1, 170, 54, 4]),
 (17, [0, 1, 163, 57, 1]),
 (18, [0, 1, 165, 47, 2]),
 (19, [0, 1, 169, 58, 1]),
 (20, [0, 1, 164, 60, 2]),
 (21, [0, 1, 168, 60, 1]),
 (22, [0, 1, 163, 50, 1]),
 (23, [0, 1, 150, 38, 1]),
 (24, [0, 1, 165, 50, 1]),
 (25, [0, 1, 165, 55, 1]),
 (26, [0, 1, 170, 68, 2]),
 (27, [0, 1, 168, 57, 1]),
 (28, [0, 1, 158, 44, 2]),
 (29, [0, 1, 160, 50, 1]),
 (30, [0, 1, 160, 50, 1]),
 (31, [0, 1, 163, 52, 1]),
 (32, [0, 1, 169, 49, 1]),
 (33, [0, 1, 172, 52, 1]),
 (34, [0, 1, 160, 48, 2]),
 (35, [0, 1, 164, 47, 1]),
 (36, [0, 1, 167, 60, 1]),
 (37, [0, 1

In [13]:
cloth_features_source = [(cloth_meta['newClothId'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i], cloth_meta['clothRate'][i], cloth_meta['clothReviewCnt'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i], cloth_meta['userMale'][i], cloth_meta['userFemale'][i], ]) for i in range(cloth_meta.shape[0])]
cloth_features_source

[(161812522, [0, 1, 4.6, 10, 165.0, 54, 0.0, 1.0]),
 (161812521, [0, 1, 4.6, 10, 162.0, 60, 0.0, 1.0]),
 (185310349, [0, 1, 5.0, 1, 167.3333333333, 57, 0.0, 1.0]),
 (1884867110, [0, 1, 5.0, 2, 167.0, 56, 0.0, 1.0]),
 (1629303146, [0, 1, 4.8, 16, 162.0, 44, 0.0, 1.0]),
 (2350864193, [0, 1, 5.0, 1, 164.0, 51, 0.0, 1.0]),
 (2193959302, [0, 1, 5.0, 9, 172.0, 56, 0.0, 1.0]),
 (1075608305, [0, 1, 4.7, 7, 158.0, 65, 0.0, 1.0]),
 (1075608306, [0, 1, 4.7, 7, 165.0, 52, 0.0, 1.0]),
 (1235557318, [0, 1, 5.0, 2, 157.5, 44, 0.0, 1.0]),
 (1287687338, [0, 1, 4.3, 22, 158.5, 54, 0.0, 1.0]),
 (1478938343, [0, 1, 4.5, 2, 161.8, 49, 0.0545454545, 0.9454545455000001]),
 (1966796358, [0, 1, 4.9, 8, 160.0, 54, 0.0, 1.0]),
 (1966796359, [0, 1, 4.9, 8, 161.25, 52, 0.0, 1.0]),
 (1963300376, [0, 1, 5.0, 2, 168.0, 56, 0.0, 1.0]),
 (1907111404, [0, 1, 4.6, 8, 168.0, 55, 0.0, 1.0]),
 (1907111405, [0, 1, 4.6, 8, 160.0, 48, 0.0, 1.0]),
 (1907111403, [0, 1, 4.6, 8, 158.5, 46, 0.0, 1.0]),
 (2017517424, [0, 1, 4.5, 15,

In [14]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['newClothId'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [15]:
interactions, weights = dataset.build_interactions(transaction_source)

In [16]:
interactions

<8297x2654 sparse matrix of type '<class 'numpy.int32'>'
	with 10311 stored elements in COOrdinate format>

In [17]:
weights

<8297x2654 sparse matrix of type '<class 'numpy.float32'>'
	with 10311 stored elements in COOrdinate format>

In [18]:
user_features = dataset.build_user_features(user_features_source)
user_features

<8297x8297 sparse matrix of type '<class 'numpy.float32'>'
	with 44181 stored elements in Compressed Sparse Row format>

In [19]:
cloth_features = dataset.build_item_features(cloth_features_source)
cloth_features

<2654x3347 sparse matrix of type '<class 'numpy.float32'>'
	with 18402 stored elements in Compressed Sparse Row format>

In [20]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()

In [32]:
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)),
         hp.uniform('learning_rate', 0.01, 0.05)]

In [33]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=30,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [34]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 20, lrn_rate: 0.01764, precision: 0.01073    
no_comp: 10, lrn_rate: 0.04872, precision: 0.01073                                 
no_comp: 20, lrn_rate: 0.02389, precision: 0.01145                                 
no_comp: 20, lrn_rate: 0.03064, precision: 0.01207                                 
no_comp: 10, lrn_rate: 0.03399, precision: 0.01073                                 
no_comp: 10, lrn_rate: 0.01968, precision: 0.01145                                 
no_comp: 30, lrn_rate: 0.04580, precision: 0.01145                                 
no_comp: 40, lrn_rate: 0.04370, precision: 0.01093                                 
no_comp: 30, lrn_rate: 0.01080, precision: 0.01135                                 
no_comp: 40, lrn_rate: 0.01814, precision: 0.01093                                 
100%|██████████| 10/10 [00:46<00:00,  4.63s/trial, best loss: -0.012068076990544796]


In [35]:
best_params

{'learning_rate': 0.03064248408861947, 'no_components': 1}

In [36]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.031 1


In [37]:
model = LightFM(no_components=no_components,
    learning_schedule='adagrad',
    loss='warp',
    learning_rate=learning_rate,
    random_state=0)

model.fit(interactions=train,
    item_features=cloth_features,
    user_features=user_features,
    sample_weight=train_weights,
    epochs=10,
    verbose=False)

In [66]:
# from lightfm import get_item_representations

# Find Similar Items
item_biases, item_embeddings = model.get_item_representations(cloth_features)
user_biases, user_embeddings = model.get_user_representations(user_features)

def make_best_items_report(item_embeddings, book_id, num_search_items=10):
    item_id = book_id - 1

    # Cosine similarity
    scores = item_embeddings.dot(item_embeddings[item_id])  # (10000, )
    item_norms = np.linalg.norm(item_embeddings, axis=1)    # (10000, )
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    print(np.argpartition(scores, -num_search_items))
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[item_id]),
                                        key=lambda x: -x[1])

    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame()

    for similar_item_id, score in similar_item_id_and_scores:
        clothId = cloth_meta.iloc[similar_item_id].newClothId
        best_items = pd.concat([best_items, cloth_meta[cloth_meta.newClothId==clothId]])

    return best_items

In [67]:
make_best_items_report(item_embeddings, 1000 , 10)

[ 598    0    2 ... 1401  367    5]


,newClothId,clothMale,clothFemale,clothRate,clothReviewCnt,userHeight,userWeight,userMale,userFemale
1650,19225331456,0,1,4.8,25,162.333333,52,0.000000,1.000000
2281,23533432151,0,1,5.0,2,163.000000,51,0.000000,1.000000
1829,11590381626,0,1,4.5,44,167.000000,58,0.000000,1.000000
1921,21716481542,0,1,5.0,1,171.000000,51,0.000000,1.000000
1725,15062541280,0,1,4.8,18,168.000000,58,0.000000,1.000000
1020,1173638783,0,1,4.8,105,168.000000,58,0.000000,1.000000
1925,21512621617,0,1,4.8,11,165.000000,52,0.000000,1.000000
1401,17957661425,0,1,4.5,8,162.766667,53,0.016667,0.983333
367,2336536292,0,1,5.0,1,166.333333,58,0.000000,1.000000
5,2350864193,0,1,5.0,1,164.000000,51,0.000000,1.000000


In [69]:
cloth_meta.iloc[1000].newClothId

1173614661.0